In [1]:
!pip install opendatasets --quiet
!pip install replicate
!pip install langchain_community


import os
import pandas as pd
import opendatasets as od

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
# =============================================
# 2. Download Dataset dari Kaggle (UFC)
# =============================================
# Dataset: https://www.kaggle.com/datasets/jerzyszocik/ufc-fight-forecast-complete-gold-modeling-dataset
od.download("https://www.kaggle.com/datasets/jerzyszocik/ufc-fight-forecast-complete-gold-modeling-dataset")

# Baca file CSV utama
ufc_df = pd.read_csv("ufc-fight-forecast-complete-gold-modeling-dataset/UFC_full_data_golden.csv")
print("Preview data UFC:")
print(ufc_df.head())

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: dimsk69
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/jerzyszocik/ufc-fight-forecast-complete-gold-modeling-dataset


100%|██████████| 55.2M/55.2M [00:00<00:00, 747MB/s]

Preview data UFC:
                             event_name          referee              winner  \
0  UFC Fight Night: Imavov vs. Borralho  Patricio Carlos  Rinat Fakhretdinov   
1  UFC Fight Night: Imavov vs. Borralho        Herb Dean      Kaue Fernandes   
2  UFC Fight Night: Imavov vs. Borralho   Lukasz Bosacki          Sam Hughes   
3  UFC Fight Night: Imavov vs. Borralho  Patricio Carlos            Oumar Sy   
4  UFC Fight Night: Imavov vs. Borralho   Lukasz Bosacki       William Gomis   

   num_rounds  title_fight         weight_class gender                result  \
0           3        False         Welterweight      M                KO/TKO   
1           3        False          Lightweight      M                KO/TKO   
2           3        False  Women's Strawweight      F            Submission   
3           3        False    Light Heavyweight      M                KO/TKO   
4           3        False        Featherweight      M  Decision - Unanimous   

                    

In [3]:
# =============================================
# 3. Preprocessing Dataset
# =============================================
# Ambil kolom yang relevan sesuai struktur dataset
cols = [
"event_name", # Nama event UFC
"winner", # Nama pemenang
"weight_class", # Kelas berat
"gender", # Gender fighter
"result", # Tipe hasil (KO/TKO, Submission, Decision)
"finish_round" # Round berakhir
]


ufc_df = ufc_df[cols].dropna().head(20)


print("\nSample data yang akan dianalisis:")
print(ufc_df)


Sample data yang akan dianalisis:
                              event_name              winner  \
0   UFC Fight Night: Imavov vs. Borralho  Rinat Fakhretdinov   
1   UFC Fight Night: Imavov vs. Borralho      Kaue Fernandes   
2   UFC Fight Night: Imavov vs. Borralho          Sam Hughes   
3   UFC Fight Night: Imavov vs. Borralho            Oumar Sy   
4   UFC Fight Night: Imavov vs. Borralho       William Gomis   
5   UFC Fight Night: Imavov vs. Borralho       Sam Patterson   
6   UFC Fight Night: Imavov vs. Borralho  Modestas Bukauskas   
7   UFC Fight Night: Imavov vs. Borralho  Nassourdine Imavov   
8   UFC Fight Night: Imavov vs. Borralho      Robert Bryczek   
9   UFC Fight Night: Imavov vs. Borralho  Benoit Saint Denis   
10  UFC Fight Night: Imavov vs. Borralho         Ante Delija   
11  UFC Fight Night: Imavov vs. Borralho           Axel Sola   
12  UFC Fight Night: Imavov vs. Borralho         Mason Jones   
13     UFC Fight Night: Walker vs. Zhang          Sumudaerji   
14   

In [4]:
# =============================================
# 4. Setup Replicate API untuk IBM Granite
# =============================================
from google.colab import userdata
api_token = userdata.get("api_token")
os.environ["REPLICATE_API_TOKEN"] = api_token


from langchain_community.llms import Replicate
llm = Replicate(model="ibm-granite/granite-3.3-8b-instruct")

In [5]:
# =============================================
# 5. Classification dengan Granite
# =============================================
results = []

for idx, row in ufc_df.iterrows():
    prompt = (
        f"""
Klasifikasikan hasil pertarungan UFC berikut ke dalam kategori:
- KO/TKO
- Submission
- Decision
Balas hanya dengan label di atas.

Event: {row['event_name']}
Pemenang: {row['winner']} (Gender: {row['gender']}, Weight Class: {row['weight_class']})
Result: {row['result']} (Round {row['finish_round']})
"""
    )
    label = llm.invoke(prompt)
    results.append({
        "event": row['event_name'],
        "winner": row['winner'],
        "weight_class": row['weight_class'],
        "gender": row['gender'],
        "result": row['result'],
        "finish_round": row['finish_round'],
        "classification": label
    })

classified_df = pd.DataFrame(results)
print("\nHasil klasifikasi Granite:")
print(classified_df.head())


Hasil klasifikasi Granite:
                                  event              winner  \
0  UFC Fight Night: Imavov vs. Borralho  Rinat Fakhretdinov   
1  UFC Fight Night: Imavov vs. Borralho      Kaue Fernandes   
2  UFC Fight Night: Imavov vs. Borralho          Sam Hughes   
3  UFC Fight Night: Imavov vs. Borralho            Oumar Sy   
4  UFC Fight Night: Imavov vs. Borralho       William Gomis   

          weight_class gender                result  finish_round  \
0         Welterweight      M                KO/TKO             1   
1          Lightweight      M                KO/TKO             1   
2  Women's Strawweight      F            Submission             2   
3    Light Heavyweight      M                KO/TKO             1   
4        Featherweight      M  Decision - Unanimous             3   

  classification  
0         KO/TKO  
1         KO/TKO  
2     Submission  
3         KO/TKO  
4       Decision  


In [8]:
# =============================================
# 6. Summarization dengan Granite
# =============================================
summary_prompt = f"""
Buat ringkasan analisis dari data klasifikasi UFC berikut:
{classified_df}


Tuliskan insight utama dalam bahasa Indonesia, buat dalam bentuk paragraf."""


summary = llm.invoke(summary_prompt)
print("\nRingkasan Granite:")
print(summary)


Ringkasan Granite:
Berdasarkan data klasifikasi UFC yang diberikan, dapat disimpulkan beberapa insight utama tentang perlombaan UFC ini. Pertama, perlombaan tersebut melibatkan pertempuran berbeda-beda di antara atlet pria (M) dan wanita (F) dalam berbagai kelas berat yang berbeda, seperti Welterweight, Lightweight, Women's Strawweight, Light Heavyweight, Featherweight, Heavyweight, dan Bantamweight.

Kedua, tingkat kemenangan yang paling sering diperoleh oleh atlet-atlet tersebut melalui KO/TKO (Knockout/Technical Knockout), yang terjadi sebanyak 12 kali. Ini menunjukkan bahwa perlombaan ini sering kali diakhiri dengan pemenang yang mencapai kemenangan dengan mengusir musuhnya dari arena.

Ketiga, beberapa perlombaan juga berakhir dengan keputusan pengadil (Decision), sebanyak 10 kali, yang mengindikasikan bahwa sifat keterampilan dan strategi yang baik, serta kemampuan untuk meneruskan perlombaan sampai ke akhir juga dianggap penting dalam UFC.

Selanjutnya, dapat dilihat bahwa kela